In [1]:
import numpy as np

KeyboardInterrupt: 

In [2]:
import pandas as pd
import numpy as np
import os

# Directory containing your files
data_dir = "output/"

# Load just one parquet file
sim_file = "10000_events_simset_1600_seed_1000_20250516_112519.parquet"
file_path = os.path.join(data_dir, sim_file)

print(f"Loading {sim_file}...")
df = pd.read_parquet(file_path)
print(f"Loaded {len(df)} events")

# Identify CC events
cc_indices = []
for i, (idx, row) in enumerate(df.iterrows()):
    if row['mc_truth']['interaction'] == 1:  # CC event
        cc_indices.append(idx)

print(f"Found {len(cc_indices)} CC events")

# Take 5 CC events for detailed analysis
sample_indices = cc_indices[:5]
print(f"Analyzing 5 sample CC events: {sample_indices}")


Loading 10000_events_simset_1600_seed_1000_20250516_112519.parquet...


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:

# Function to calculate inelasticity
def analyze_event_inelasticity(event_data):
    """Calculate inelasticity for a CC event"""
    results = {}
    
    # Get neutrino energy from mc_truth
    if 'primary_energy' in event_data['mc_truth']:
        neutrino_energy = event_data['mc_truth']['primary_energy']
    else:
        neutrino_energy = None
        print("  Warning: Primary energy not found")
    
    results['neutrino_energy'] = neutrino_energy
    
    # Get final state particles
    final_state_particles = event_data['mc_truth']['final_state_type']
    results['final_state_particles'] = final_state_particles.tolist() if isinstance(final_state_particles, np.ndarray) else final_state_particles
    
    # Check if final state energies are available
    if 'final_state_energy' in event_data['mc_truth']:
        final_state_energies = event_data['mc_truth']['final_state_energy']
        results['final_state_energies'] = final_state_energies.tolist() if isinstance(final_state_energies, np.ndarray) else final_state_energies
    else:
        final_state_energies = None
        print("  Warning: Final state energies not found")
    
    # Look for muons in the final state
    muon_indices = []
    if isinstance(final_state_particles, np.ndarray):
        muon_indices = np.where(np.abs(final_state_particles) == 13)[0]
    
    results['muon_indices'] = muon_indices.tolist() if isinstance(muon_indices, np.ndarray) else muon_indices
    
    # If no muons found, inelasticity can't be calculated
    if len(muon_indices) == 0:
        results['muon_energy'] = None
        results['inelasticity'] = None
        results['status'] = "No muons found"
        return results
    
    # If multiple muons, find the highest energy one
    if len(muon_indices) > 1:
        if final_state_energies is not None:
            # Get energies of the muons
            muon_energies = [final_state_energies[i] for i in muon_indices]
            # Find the most energetic muon
            max_energy_idx = np.argmax(muon_energies)
            muon_idx = muon_indices[max_energy_idx]
            muon_energy = muon_energies[max_energy_idx]
            results['status'] = f"Multiple muons, using highest energy: {muon_energy}"
        else:
            # If energies not available, use the first muon
            muon_idx = muon_indices[0]
            muon_energy = None
            results['status'] = "Multiple muons, energies not available, using first muon"
    else:
        # Just one muon
        muon_idx = muon_indices[0]
        if final_state_energies is not None:
            muon_energy = final_state_energies[muon_idx]
            results['status'] = f"Single muon with energy: {muon_energy}"
        else:
            muon_energy = None
            results['status'] = "Single muon, energy not available"
    
    results['muon_energy'] = muon_energy
    
    # Calculate inelasticity if both energies are available
    if neutrino_energy is not None and muon_energy is not None:
        inelasticity = 1.0 - (muon_energy / neutrino_energy)
        results['inelasticity'] = inelasticity
    else:
        results['inelasticity'] = None
    
    # Try to get muon energy from photons if not available
    if muon_energy is None and 'photons' in event_data:
        # Get photon data
        photons = event_data['photons']
        
        # Get id_idx for mapping to particle types
        id_idx_array = photons['id_idx']
        
        # Map to particle types
        particle_types = []
        for idx in id_idx_array:
            if idx > 0 and idx <= len(final_state_particles):
                particle_types.append(final_state_particles[idx-1])
            else:
                particle_types.append(-1)
        
        # Count hits from muons
        muon_mask = np.array(particle_types) == 13
        muon_hit_count = np.sum(muon_mask)
        results['muon_hit_count'] = int(muon_hit_count)
        
        # If the primary muon is identified by id_idx, we could estimate energy from hits
        # This would require a calibration model relating hit count to energy
        # For now, just report the hit count as a proxy
    
    return results

# Process the sample events
for i, event_idx in enumerate(sample_indices):
    print(f"\nEvent {event_idx}:")
    event_data = df.iloc[event_idx]
    results = analyze_event_inelasticity(event_data)
    
    # Print detailed results
    print(f"  Neutrino energy: {results['neutrino_energy']} GeV")
    print(f"  Final state particles: {results['final_state_particles']}")
    
    if 'final_state_energies' in results:
        print(f"  Final state energies: {results['final_state_energies']}")
    
    print(f"  Muon indices: {results['muon_indices']}")
    print(f"  Status: {results['status']}")
    print(f"  Muon energy: {results['muon_energy']} GeV")
    
    if results['inelasticity'] is not None:
        print(f"  Inelasticity (y): {results['inelasticity']:.4f}")
    else:
        print(f"  Inelasticity: Cannot calculate (missing energy information)")
    
    if 'muon_hit_count' in results:
        print(f"  Muon hits: {results['muon_hit_count']}")